# Date

This notebook detects Date annotations as an initial step. We start with Dates because they are probably the easiest annotations and they can also be useful for the detection of the other annotations.

The first cell contains the rules for the detection of Dates. The input is set to the training dcouments. These files (XMIs) contain complete reference section and also gold annoations for Author, Date, Title, Venue and Reference. We configure the cell to evaluate Date annotations and to display the evaluation results. Additionally, we store the content of the cell as a Ruta script and also the corresponding type system. 

Try to remove some rules and investigate the effect...

In [ ]:
%inputDir data/train
%outputDir out_date_train
%displayMode EVALUATION
%evalTypes Date
%writescript ./Date.ruta
%saveTypeSystem ./DateTypeSystem.xml

TYPESYSTEM ReferencesTypeSystem;

DECLARE MonthInd, YearInd;
WORDLIST MonthList = "months.txt";
MARKFAST(MonthInd, MonthList);

DECLARE NoDatePrefix;
WORDLIST NoDatePrefixList = "no_date_prefix.txt";
MARKFAST(NoDatePrefix, NoDatePrefixList);

BLOCK(Date) Reference{}{
    
    // some simple candidates for dates
    NUM{REGEXP("19..|20..")-> YearInd};
    // some dates also have an additional char
    y:@YearInd{-> y.end=sw.end} sw:SW{REGEXP("[abc]")};
    
    // create dates using YearInd
    (SPECIAL.ct=="(" @YearInd SPECIAL.ct==")" COMMA?){-> Date};
    (@YearInd{-PARTOF(Date)} PERIOD){-> Date};
    (@YearInd{-PARTOF(Date)} COMMA[0,2]){-> Date};
    
    // expand Dates based on context
    m:MonthInd (NUM SPECIAL)? NUM COMMA? d:@Date{-> d.begin = m.begin};
    s:NUM SPECIAL NUM MonthInd d:@Date{-> d.begin = s.begin};
    m:MonthInd d:@Date{-> d.begin = m.begin};
    w1:W{INLIST(MonthList, ""+w1.ct+w2.ct)} SPECIAL w2:W d:@Date{-> d.begin = w1.begin};
    
    s1:SPECIAL.ct=="(" d:@Date{-> d.begin=s1.begin,d.end=s2.end} s2:SPECIAL.ct==")";
    d:@Date{-> d.end=e.end} e:PERIOD;
    b:NUM{REGEXP(".|[12].|3[01]")} d:@Date{STARTSWITH(YearInd)-> d.begin=b.begin};
    
    // remove some false positive dates
    NoDatePrefix SPECIAL? d:@Date{-> UNMARK(d)};
    d1:Date{-> UNMARK(d1)} SPECIAL NUM;
    NUM SPECIAL d2:@Date{-> UNMARK(d2)};
}

// if there are several dates in one reference, try to remove some potetially false positive
Reference{CONTAINS(Date,2,100)} -> {
    CAP SPECIAL? d:@Date{-> UNMARK(d)};
};
Reference{CONTAINS(Date,2,100)} -> {
     d:@Date{-> UNMARK(d)} SPECIAL? CAP;
};


In the second cell, we use the results of the previous cell, i.e. the CAS files in the out_train folder, to investigate the remaining problems. This is achived by setting the display mode CSV to summarize BadReferences, spans that contains a evaluation error (FalsePrositive or FalseNegative). The evaluation annotatitons are highlghted using different colors.

In [ ]:
%inputDir out_date_train
%outputDir trash
%displayMode CSV
%csvConfig BadReference

DECLARE BadReference;
Reference{OR(CONTAINS(FalsePositive),CONTAINS(FalseNegative)),-PARTOF(BadReference)-> BadReference};

COLOR(TruePositive, "lightgreen");
COLOR(FalsePositive, "lightblue");
COLOR(FalseNegative, "pink");

In this cell, we load and apply the rules of the first cell (Date.ruta) on the validation documents. Thus, we can investigate how well the rules generalize to other examples. The evaluation results are displayed as before and the resulting annotated CAS files are stored in the folder out_validation.

In [ ]:
%inputDir data/validation
%outputDir out_date_validation
%displayMode EVALUATION
%evalTypes Date

SCRIPT Date;
CALL(Date);

Now, we investigate the falsely annotated Dates in the validation set.

In [ ]:
%inputDir out_date_validation
%outputDir trash
%displayMode CSV
%csvConfig BadReference

DECLARE BadReference;
Reference{OR(CONTAINS(FalsePositive),CONTAINS(FalseNegative)),-PARTOF(BadReference)-> BadReference};

COLOR(TruePositive, "lightgreen");
COLOR(FalsePositive, "lightblue");
COLOR(FalseNegative, "pink");

Finally, we also evaluate the rules on the test set but without investigating the resulting annotations.

In [ ]:
%inputDir data/test
%displayMode EVALUATION
%evalTypes Date

SCRIPT Date;
CALL(Date);